In [ ]:
!pip install refinery==0.0.4 --index-url=https://artifactory.prd.it.unity3d.com/artifactory/api/pypi/pypi/simple

In [1]:
from unity_vision.clients import UCVDClient
from unity_vision.consumers.solo.parser import Solo

In [2]:
client = UCVDClient(
    project_id="3e3b0fb5-50fe-4802-a9eb-db81bae8d185",
    org_id="1374969834147",
    sa_key="a5ba312e-e036-4f79-b0f5-16b18bf8c408",
    api_secret="Nzvy_DRa-clMH0iXZoi4VvCPtd2vrQsw"
)

dest_dir="solo_dir"
dataset_id="803f11ec-cd02-4625-9a67-9b4b0aafca4b"


In [3]:

archives = client.list_dataset_archives(dataset_id)
# client.download_dataset_archives(dataset_id=dataset_id, dest_dir="solo_dir", skip_on_error=False)

# TODO: Add API to extract and inflate all archives to one dataset at solo/

In [ ]:
solo = Solo(f"{dest_dir}/solo")

In [ ]:
print("Dataset Metadata !! ")
print(solo.get_metadata())

In [ ]:
while True:
    try:
        frame = next(solo)
        annotations, metrics = [cap.annotations for cap in frame.captures], frame.metrics
        print(annotations)
        print(metrics)
    except StopIteration:
        break

In [ ]:
!refinery local ./pipeline.yml